### 11.1 모나드 이해하기
* `모나드 Monad는 수학의 카테고리인론이라는 분야에서 사용되는 용어`이다.
* 모나드는 `일정의 코드설계패턴으로서 몇 개의 인터페이스를 구현한 클래스`

#### 타입클래스란?
* 모나드를 이해하는 것은 타입클래스가 왜 필요한지를 아는 것
* const callMap = fn => b => b.map(fn) 
* calMap은 b가 map이라는 메서드를 가진 2차고차함수
  - callMap = {fn => { return  (b => { return b.map(fn) } ) }
* 따라서 아래 코드는 정상적으로 실행된다.
* callMap(a => a + 1)([1]) // 정상종료
* 하지만 작성자의 의도를 이해하지 못하면 비정상적으로 종료된다.
* callMap(a => a + 1)(1) // 비정상종료
* 이를 방지하려면 다음처럼 매개변서 b는 반드시 map메서드가 있는 타입이라고 타입을 제한해야 한다.
* `const callMap = <T, U>(fn: (T) => U) => <T extends {map(fn)}>(b: T) => b.map(fn)`
  - callMap 상세설명참고
  
* 보통 객체지향언어라면 Number라는 클래스를 만들고 map메서드를 구현하는 식으로 구현하겠지만
* `모나드식설게는 반드시 map과 of라는 이름의 메서드가 있는 Monad<T>클래스를 만든다.`

```ts
class Monad<T> {
  constructor(public valueL T) {}
  static of<U>(value: U):Monad<U> { return new Monad<U>(value) }
  map<U>(fn: (x: T) => U):Monad<U> { return new Monad<U>(fn(this.value))}
}
```
* 이처럼 Monad<T>와 같은 클래스를 타입클래스라고 한다. 타입클래스는 아래와 같이
* 함수를 만들 때 특별한 타입으로 제약하지 않아도 된다.
* `const callManad = (fn) => (b) => Monad.of(b).map(fn).value`
* Monad<T>와 같은 타입클래스 덕분에 callMonad처럼 타입에 따른 안정성을 보장하면서도
* 코드의 재사용성(code reusability)이 뛰어난 범용함수를 만들 수 있다.
```ts
callMonad((a:Number) => a+1)(1)
callMonad((a:Number[]) => a.map(value => value + 1))([1,2,3,4])
```
    
#### 고차타입이란?
* Monad<T>처럼 타입 T를 한 단계 더 높은 타입으로 변환하는 용도의 타입을 고차타입이라한다.
* 고차타입(higher-kinded type)은 카테고리이론이라는 수학에서 아이디어를 얻었다.
    
#### 카테고리 이론이란?
* 카테고리이론은 1940년대에 시작된 수학의 한 분야.
* 함수형 프로그래밍 언어의 중요한 이론적인 배경이 되었다.
* 수학에서 set(집합)은 프로그래밍에서 타입이다
* 수학에서 카테고리는 `집합의 집합`으로 이해할 수 있다.
* 프로그래밍에서 카테고리는 타입의 타입, 즉 고차타입으로 이해할 수 있다.
* 그리고 모나드는 별동의 특징이 있는 고차타입이다.
    
#### 판타지랜드 규격
* 모나드는 원래 카테고리 이론에서 사용되는 용어였지만, 함수형 프로그래밍 언어의 최고봉인 
* 하스켈 언어의 Prelude라는 표준 라이브러리에서 사용되는 용어이기도 하다.
* 모나드는 '모나드 룰 Monad raw'이라고 하는 코드설계원칙에 맞춰 구현된 클래스를 의미한다.
* 판타지랜드 규격이란 `하스켈 표준 라이브러리 구조를 자바스크립티방식으로 재구성한 것`이다.
* 특정 클래스가 다음 4가지 조건을 만족하면 그 클래스는 모나드이다.
  1. 펑터(Functor) : `map이라는 인스턴스 메서드`를 가지는 클래스
  2. 어플라이(Apply) : `펑터이면서 ap라는 인스턴스 메서드`를 가지는 클래스
  3. 애플리커티브(Applicative) : `어플라이면서 of라는 인스턴스 메서드`를 가지는 클래스
  4. 체인(Chain) : 애플리커티브이면서 `chain이라는 인스턴스 메서드`를 가지는 클래스

#### 모나드 룰
* 어떤 클래스의 이름이 M이고 이 클래스의 인스턴스를 m이라고 할 때
* 모나드는 앞서 언급한 애플리커티브와 체인의 기능을 가지고 있고
* 다음과 같은 2가지 법칙을 만족하게 구현한 클래스이다.
* 모나드의 룰의 왼쪽 법칙과 오른쪽 법칙
    
|구분|의미|
|:-------:|:--------:|
|왼쪽 법칙(left identify)|M.of.chain(f) == f(a)|
|오른쪽 법칙(right identify)|m.chain(M.of) == m|

#### callMap 상세설명

```ts
// 코드 분석 및 주석 추가
// 1. callMap 함수: `fn`을 받아서 `map(fn)`을 호출하는 함수를 반환
const callMap = <T, U>(
// `fn`은 입력 `T`를 받아 출력 `U`를 반환하는 함수
fn: (arg: T) => U
) => 
// 2. 이 부분은 고차 함수 (함수를 반환하는 함수)
<T extends { map(fn: (arg: any) => any): any }>(
// `b`는 반드시 `map` 메서드를 가진 객체여야 한다. (ex: 배열)
b: T
) => 
// 3. `b.map(fn)`을 호출하여 결과 반환
b.map(fn);
```

##### 각 부분 자세한 설명
```ts
/* 

1) callMap 함수의 타입 정의

🔍 설명

<T, U>(fn: (arg: T) => U)

  * <T, U>:
    - 제네릭 타입 T: 입력 데이터의 타입
    - 제네릭 타입 U: 변환된 데이터의 타입
  * fn: (arg: T) => U:
    - fn은 T 타입을 받아서 U 타입을 반환하는 함수
    - 즉, 변환 함수(ex: number를 string으로 변환하는 함수 등)를 의미

📌 예제
  const toString = (x: number): string => x.toString();
  ... 여기서 toString의 타입은 (number) => string이므로, T = number, U = string이 됩니다.

2) map을 가진 객체(b)를 받는 부분

<T extends { map(fn: (arg: any) => any): any }>

🔍 설명
  * 이 부분은 map 메서드를 가진 객체(예: Array)만 받을 수 있도록 제한합니다.
  * T extends { map(fn: (arg: any) => any): any }
    - T는 반드시 map 메서드를 가진 타입이어야 한다.
    - 즉, T는 배열과 같은 객체이어야 한다.
    - map(fn: (arg: any) => any): any
      - map은 fn을 받아서 어떤 값을 반환하는 함수여야 한다.
      - 배열의 map 메서드와 동일한 형태를 가짐.
      
📌 예제
const numbers = [1, 2, 3];
numbers.map((x) => x * 2); // 가능 ✅
위 코드에서 numbers는 map을 가진 객체이므로 T 조건을 만족합니다.

❌ map이 없는 객체를 전달하면 오류 발생:
const obj = { a: 1, b: 2 };
callMap((x: number) => x * 2)(obj); // ❌ 오류 발생
* obj는 map 메서드를 가지고 있지 않으므로 타입 오류가 발생합니다.

3) 최종적으로 b.map(fn) 실행

b.map(fn);

* b는 반드시 map을 가진 객체(Array 등)여야 하므로, map을 안전하게 호출할 수 있습니다.
* map(fn)을 실행하면 각 요소가 fn을 통해 변환됩니다.
📌 예제
const numbers = [1, 2, 3];
const double = callMap((x: number) => x * 2);
console.log(double(numbers)); // [2, 4, 6]

1. callMap((x: number) => x * 2) 실행:
   - fn = (x: number) => x * 2
   - callMap은 새로운 함수를 반환함.
2. double(numbers) 실행:
   - b = [1, 2, 3]
   - b.map(fn) → [2, 4, 6]
```

| 부분 | 설명 |
|:------|:------|
| `<T, U>(fn: (arg: T) => U)` | `T`를 받아 `U`로 변환하는 함수를 받음 |
| `<T extends { map(fn: (arg: any) => any): any }>(b: T)` | `map` 메서드를 가진 객체만 받을 수 있음 |
| `b.map(fn)` | `b`의 각 요소에 대해 `fn`을 적용한 새 배열을 반환 |


### 11.2 Identify 모나드 이해와 구현

#### 실습프로젝트 구성

```bash
mkdir 11.monad
cd 11.monad
npm --init -y
npm i -D typescript ts-node @types/node

tsc --init

mkdir src/1102/test
mkdir src/1102/interfaces
mkdir src/1102/classes
```

#### 값 컨테이너 구현용 IValuable<T> 인터페이스 구현
* 어떤 타입T가 있을 때 배열 T[]는 같은 타입의 아이템을 여러개 가진 컨터이너이다.
* 이처럼 타입 T를 가지는 값의 컨테이너를 '값 컨테이너 value container'라고 한다.
* 여러개의 interface를 구현하는 Identity<T>클래스를 구현하는 step
    
###### src/1102/interfaces/IValuable.ts
    
```ts
export interface IValuable<T> {
  value(): T
}
```
    
###### src/1102/interfaces/index.ts
    
```ts
import {IValuable} from './IValuable'
import {ISetoid} from './ISetoid'
import {IFunctor} from './IFunctor'
import {IApplicative} from './IApplicative'
import {IApply} from './IApply'
import {IChain} from './IChain'
import {IMonad} from './IMonad'

export {IValuable, ISetoid, IFunctor, IApplicative, IApply, IChain, IMonad}
```  
    
#### 클래스 이름이 왜 Identity인가?
* 함수형 프로그래밍에서 identity는 항상 다음처럼 구현하는 특별한 의미의 함수이다.
  - `const identity = <T>(value: T): T => value`
* `자신의 타입에서 다른 타입으로 갔다가 돌아올 떄 값이 변경되지 않은 카테고리를 identity`라고 한다. 
  - 카테고리에서 idenity의미, 아래 코드는 Identity의 예
  - `Identity.od(1).chain(identity.of)  // Identity.of(1)`
    
#### 값 컨테이너로서의 Identity<T> 구현하기
* ISetoid<T>      : equals메서드 제공
* IFunctor<T>     : map 메서드 제공 
* IApply<T>       : ap 메서드 제공
* IChain<T>       : cahin 메서드 제공    
* IApplicative<T> : of 메서드 제공        
    
###### src/1102/classes/Identity.ts
    
```ts
import {IMonad, ISetoid} from '../interfaces'

export class Identity<T> implements ISetoid<T>, IMonad<T> {
  constructor(private _value: T) {}
  value() { return this._value }

  static of<T>(value: T): Identity<T> { return new Identity<T>(value)}

  equals<U>(that: U): boolean {
    if(that instanceof Identity)
      return this.value() == that.value()
    return false
  }

  // IFunctor
  map<U, V>(fn: (x: T) => U): Identity<U> {
    return new Identity<U>(fn(this.value()))
  }

  // IApply
  ap<U>(b: U) {
    const f = this.value() 
    if(f instanceof Function)
      return Identity.of<U>((f as Function)(b))
  }

  // IChain
  chain<U>(fn: (T) => U): U {
    return fn(this.value())
  }
}
``` 

###### src/1102/test/ISetoid-test.ts
 
```ts
import {Identity} from '../classes/Identity'

const one = new Identity(1), anotherOne = new Identity(1)
const two = new Identity(2)
console.log(
  one.equals(anotherOne),  // true
  one.equals(two),  // false
  one.equals(1),    // false
  one.equals(null), // false
  one.equals([1])   // false
)
```

###### src/1102/test/IApply-test.ts
 
```ts
import {Identity} from '../classes/Identity'

const add = x => y => x + y
const id = new Identity(add)

console.log(
  id.ap(1).ap(2).value() // 
)
```

###### src/1102/test/IMap-IChain-test.ts
 
```ts
import {Identity} from '../classes/Identity'

console.log(
  Identity.of(1).map(value => `the count is ${value}`).value(), // the count is 1
  Identity.of(1).chain(value => Identity.of(`the count is ${value}`)).value() // the count is 1
)
```
    
###### src/1102/test/IMonad-left-law-test.ts
 
```ts
import {Identity} from '../classes/Identity'

const a = 1
const f = a => a * 2
console.log(
  Identity.of(a).chain(f) == f(a) // true
)
```
    
###### src/1102/test/IMonad-right-law-test.ts
 
```ts
import {Identity} from '../classes/Identity'

const m = Identity.of(1)

console.log(
 m.chain(Identity.of).equals(m), // true
)
```   
    
###### src/1102/testIMonad-method-chain-test.ts
 
```ts
import {Identity} from '../classes/Identity'
type IPerson = { name: string, age: number }
const jack = Identity.of(['Jack', 32])

console.log(
  jack
    .map( ([name, age]) => ({name, age}) )
    .chain( (p: IPerson) => Identity.of(p))
    .map( ({name, age}) => [name, age])
    .value()[0] == jack.value()[0] // true
)
```

### 11.3 Maybe 모나드 이해와 구현

#### 실습프로젝트 구성

```bash
npm i -s ramda node-fetch
npm i -D @types/ramda @types/node-fetch

mkdir src/1103/test
mkdir src/1103/interfaces
mkdir src/1103/classes
```

#### Maybe 모나드란?
* Maybe는 오류, 정상적일 떄 모두 고려하면서 사용하는 쪽의 코드를 간결하게 작성할 수 있도록 한다.
* 즉, 데이터의 유무에 따라 코드가 적절하게 동작하도록 설계되어 있다.
* Maybe는 하스켈 Prelude표준 라이브러리에서 제공하는 모나드 이다.
* Maybe는 그 자체가 모나드가 아니라 Maybe가 제공하는 Just<T>와 Nothing타입이 모나드이다.
```ts
export class Maybe<T> {
  static Just<U>(value: U) { return new Just<U>(value) }
  static Nothing = new Nothing
}
```  
* `Maybe의 설계목적은 코드의 안정성을 함수형 방식으로 보장`하기 위해서 이다.
* 코드의 적용되는 값에 따라 어떤 떄는 정상적으로, 어떤 떄는 undefined, null, Infinity등의 
* 값을 유발할 때 Maybe를 사용하면 매우 효율적인 방식으로 코드를 작성할 수 있다.
```ts
// 예를 들어 다음 코드는 b의 값이 undefined, null, 0이 아닐 떄는 Maybe.Just(a/b)가 반환
// 그 반대일 때는 Maybe.Nothing이 반환된다.
import { Maybe, IMaybe } from ' ../classes/Maybe'
  static Just<U> = (a:Number) => (b:number): IMaybe<number> 
    => b ? Maybe.Just(a/b) : Maybe.Nothing
}
    
// 따라서, devide함수를 사용하는 다음코드는 간결하면서도 안정성을 해치지 않게 작성
import * as from 'ramda'
console.log(
  divide(1)(1).map(R.add(1)).getOrElse(0),  // 2
  divide(1)(0).map(R.add(1)).getOrElse(0)   // 0
)
```

#### Maybe 클래스 구조
    
##### src/1102/classes/Maybe.ts
 
```ts
import {Just} from './Just'
import {Nothing} from './Nothing'
import {IMonad} from '../interfaces'
import {_IMaybe } from './_IMaybe'

export class Maybe<T> {
  static Just<U>(value: U) { return new Just<U>(value) }
  static Nothing = new Nothing
}

export type IMaybe<T> = _IMaybe<T> & IMonad<T>
export {Just, Nothing}
```

#### Maybe가 함수의 반환 타입일 때의 문제점
* 현재 TS버전 3.7.4에서는 Just<number> | Nothing과 같은 2클래스가 합집합타입을 만나면 에러가 발생
* 하지만 IMaybe라는 인터페이스를 함수의 반환타입으로 사용하면 정상컴파일이 된다.
* 이러한 특성 때문에 _IMaybe인터페이스와 IMonad인터페이스를 합쳐 놓은 IMaybe타입을 제공
    
##### src/1102/classes/_Maybe.ts   
    
```ts
export interface _IMaybe<T> {
  isJust(): boolean
  isNothing() : boolean
  getOrElse(defaultValue: T): T
}
```

#### Just모나드 구현
    
##### src/1102/classes/Just.ts       
    
```ts
import { IMonad } from '../interfaces'
import {_IMaybe } from './_IMaybe'

export class Just<T> implements _IMaybe<T>, IMonad<T> {
  constructor(private _value: T) {}
  value(): T { return this._value }

  // IApplicative
  static of<T>(value: T): Just<T> { return new Just<T>(value)}

  // IMaybe
  isJust() { return true }
  isNothing() { return false }
  getOrElse<U>(defaultValue: U) { return this.value() }

  // IFunctor
  map<U, V>(fn: (x: T) => U): Just<U> { return new Just<U>(fn(this.value())) }

  // IApply
  ap<U>(b: U) {
    const f = this.value() 
    if(f instanceof Function)
      return Just.of<U>((f as Function)(b))
  }

  // IChain
  chain<U>(fn: (T) => U): U { return fn(this.value()) }
}
```

#### Nothing t모나드 구현
* Nothing모나드는 Just모나드와 달리 코드를 완벽하게 실행 시키지 않는 것이 설계목적
* divide(1)(0).map(R.add(1)).getOrElse(0)형태의 코드가 비정상적으로 동작하지 않고 getOrElse가 반환
    
##### src/1102/classes/Nothing.ts    
 
```ts
import {IMonad} from '../interfaces'
import {_IMaybe } from './_IMaybe'

export class Nothing implements _IMaybe<null>, IMonad<null> {
  // IApplicative
  static of<T>(value: T = null): Nothing { return new Nothing }
  
  // IMaybe
  isJust() { return false }
  isNothing() { return true }
  getOrElse<U>(defaultValue: U) { return defaultValue }

  // IFunctor
  map<U, V>(fn: (x) => U): Nothing { return new Nothing }

  // IApply
  ap<U>(b: U) {
    return new Nothing
  }

  // IChain
  chain<U>(fn: (T) => U): Nothing { return new Nothing }
} 
```
    
#### Just와 Nothing 모나드 단위 테스트
    
##### src/1102/test/Just-test.ts
    
```ts
import * as R from 'ramda'
import {Just} from '../classes/Maybe'

console.log(
  Just.of(100).isJust(),                      // true
  Just.of(100).isNothing(),                  // false
  Just.of(100).getOrElse(1),                 // 100
  Just.of(100).map(R.identity).getOrElse(1),   // 100
  Just.of(R.identity).ap(100).getOrElse(1),     // 100
  Just.of(100).chain(Just.of).getOrElse(1)      // 100  
)
```
    
##### src/1102/test/Nothing-test.ts   
    
```ts
import * as R from 'ramda'
import {Nothing, Just} from '../classes/Maybe'

console.log( 
  Nothing.of().isJust(),                       // false
  Nothing.of().isNothing(),                    // true
  Nothing.of().getOrElse(1),                   // 1
  Nothing.of().map(x => x + 1).getOrElse(1),   // 1
  Nothing.of().ap(1).getOrElse(1),             // 1
  Nothing.of().chain(Just.of).getOrElse(1),    // 1 
)     
```

#### Maybe 테스트
* Maybe 모나드를 사용하는 예
  - 웹서버는 HTML을 서버에서 만들어 보내는 경우
  - Json데이터를 HTML로 동적으로 생성하게 하는 API서버    
* 웹브라우저에서 API서버에서 데이터를 가져올 때 fetch함수를 사용한다.
* fetch함수는 url을 전달받아 Promise객체를 반환한다.
    
##### src/1102/fetchJokes.ts   
    
```ts
import fetch from 'node-fetch'

export const fetchJokes = <T>() => new Promise<T>( (resolve, reject) => {
  const jokeUrl = 'https://api.icndb.com/jokes/random/5?limitTo=[nerdy]'
  
  fetch(jokeUrl)
    .then(res => res.json())
    .then( (fetchResult: unknown) => resolve(fetchResult as T))
    .catch((e: Error) => reject(e))
}) 
```
    
##### src/1103/test/fetchJoke-test.ts   
    
```ts
import {fetchJokes} from '../fetchJokes'

fetchJokes()
  .then(result => console.log(result))
  .catch((e: Error) => console.log(e.message))
```
    
##### src/1103/getRandomJoke.ts   
    
```ts
import { fetchJokes } from "./fetchJokes";
const random = (max: number) => Math.floor(Math.random() * max);

export type FetchResult = { type: string; value: JokeType[] };
export type JokeType = { id: number; joke: string; category: string[] };

export const getRandomJoke = () =>
  new Promise<JokeType>((resolve, reject) => {
    fetchJokes<FetchResult>()
      .then((result: FetchResult) => {
        let array: JokeType[] = result.value;
        resolve(array[random(array.length)]);
      })
      .catch((e: Error) => reject(e));
  });

```
    
##### src/1103/test/getRandomJoke-test.ts   
    
```ts
import * as R from 'ramda'
import { getRandomJoke, JokeType } from '../getRandomJoke'

getRandomJoke()
  .then((JokeItem: JokeType) => {
    // @ts-ignore
    const joke = R.view(R.lensProp('joke'))(JokeItem)
    console.log(joke) // Chuck Norris doesn't need an OS.
  })
  .catch((e: Error) => console.log(e.message))
```    
    
##### src/1103/getJokeAsMaybe.ts   
* getRandomJoke를 then...catch를 없애고 Maybe를 사용해서 수정
    
```ts
import * as R from 'ramda'
import { getRandomJoke, JokeType } from './getRandomJoke'
import { IMaybe, Maybe } from './classes/Maybe'

const _getJokeAsMaybe = async () => {
  const jokeItem: JokeType = await getRandomJoke()
  // @ts-ignore
  const joke = R.view(R.lensProp('joke'))(jokeItem)
  return joke
}

export const getJokeAsMaybe = () =>
  new Promise<IMaybe<string>>((resolve, reject) => {
    _getJokeAsMaybe()
      .then((joke: string) => resolve(Maybe.Just(joke)))
      .catch((e) => resolve(Maybe.Nothing))
  })

export { IMaybe, Maybe }
```    
        
    
##### src/1103/test/getJokeAsMaybe-test.ts   
* getRandomJoke를 then...catch를 없애고 Maybe를 사용해서 수정
    
```ts
import {getJokeAsMaybe, IMaybe} from '../getJokeAsMaybe'

(async() => {
  const joke: IMaybe<string> = await getJokeAsMaybe()
  console.log(joke.getOrElse('something wrong')) 
    // Chuck Norris doesn't use GUI, he prefers COMMAND line.
})()
``` 

### 11.4 Validation 모나드 이해와 구현

#### Validation 모나드란?
* 데이터유무에 따라 적절하게 동작하게 하는 것이 Maybe모나드 였다면
* 데이터가 있지만 유효여부를 판단하는 용도로 설계된 모나드가 Validation 모나드이다.
* Validation 모나드는 판타지랜드의 어플라이 규격에 의존해 동작한다.
* Validation 클래스는 Maybe와 비슷하게 Success와 Failure 두 가지 모나드로 구성된다.

#### src/1104/test/ap-test.ts
* Identity모나드에 2차 고차함수를 value로 삼은 뒤 ap메서드를 두 번 호출해 3을 만든다.

```ts
import {Identity} from '../classes/Identity' // 1

const add = (a:number) => (b: number) => a + b
console.log(
  add(1)(2), // 3
  Identity.of(add)
  .ap(1)
  .ap(2)
  .value() // 3
)
```

##### src/1104/getRandomJoke.ts
* N차 고차함수를 다음과 같이 만들 수 있다.
* checkSuccess함수는 두 개의 고차 매개변수들을 배열로 만든 후
* isFailure값이 true인 것만 추려내 그 개수가 0일 때만 '성공'이라 판단한다.

```ts
import { Identity } from "./classes/Identity";
type ISuccess = { isSuccess: boolean, isFailure: boolean }

const checkSuccess = (a: ISuccess) => (b: ISuccess):boolean => 
  [a, b].filter(({isFailure}) => isFailure == true).length == 0

const isSuccess = Identity.of(checkSuccess)
                          .ap({isSuccess: true, isFailure: false})
                          .ap({isSuccess: false, isFailure: true})
                          .value()
console.log(isSuccess)  // false
)
```

#### Validation 클래스 구조

##### src/1104/classess/Validation.ts

```ts
import {Success} from './Success'
import {Failure} from './Failure'

export class Validation {
  static Success = Success
  static Failure = Failure
  static of<T>(fn: T): Success<T> { return Success.of<T>(fn)}
}
export {Success, Failure}
```
##### src/1104/classess/IValidation.ts

```ts
export interface IValidation<T> {
  isSuccess: boolean
  isFailure: boolean
}
```

#### Success 모나드 구현
* Success모나드는 IChain형태로 동작하지 않기 때문에 IFunctor, IApply, IApplicative만 구현

##### src/1104/classess/Success.ts

```ts
import {IFunctor, IApply}from '../interfaces'
import {IValidation} from './IValidation'

export class Success<T> implements IValidation<T>, IFunctor<T>, IApply<T> { 
  constructor(public value: T, public isSuccess=true, public isFailure=false) {} 

  // IApplicative
  static of<U>(value: U): Success<U> { return new Success<U>(value)}

  // IFunctor
  map<U>(fn: (x: T) => U) { 
    return new Success<U>(fn(this.value)) 
  }
  // IApply
  ap(b) { 
    return b.isFailure ? b : b.map(this.value) 
  }
}

```

##### src/1104/test/Success-test.ts

```ts
import {Success} from '../classes/Success'

const checkSuccess = <T>(a:Success<T>) => (b:Success<T>): boolean => 
  [a, b].filter(({isFailure}) => isFailure == true).length  == 0

console.log(
  Success.of(checkSuccess)
    .ap(Success.of(1))
    .ap(Success.of(2)) // Success { value: true, isSuccess: true, isFailure: false }
)
```

#### Failure 모나드 구현

##### src/1104/classess/Failure.ts

```ts
import {IFunctor, IApply} from '../interfaces'
import {IValidation} from './IValidation'

export class Failure<T> implements IValidation<T>, IFunctor<T>, IApply<T> { 
  constructor(public value: T[], public isSuccess=false, public isFailure=true) {}

  // IApplicative
  static of<U>(value: U[]): Failure<U> { return new Failure<U>(value)}

  // IFunctor 
  map(fn) { return new Failure<T>(fn(this.value)) }
  
  // IApply
  ap(b) { 
    return b.isFailure ? new Failure<T>([...this.value, ...b.value]) : this
  }
}
```

#### 비밀번호 검증 기능 구현

##### src/1104/utils/checkNull.ts

```ts
import {Success, Failure} from '../classes/Validation'

export const checkNull = <S, F>(o: {password?: string}) => {
  const {password} = o
  return (password == undefined || typeof password != 'string') ?
    new Failure(["Password can not be null"]) : new Success(o)
}
```

##### src/1104/utils/checkLength.ts

```ts
import {Success, Failure} from '../classes/Validation'

export const checkLength = (o: {password?: string}, minLength: number = 6) => {
  const {password} = o
  return (!password || password.length < minLength) ?
    new Failure(['Password must have more than 6 characters']) : new Success(o)
}
```        

##### src/1104/checkPassword.ts

```ts
import {Success, Failure} from '../classes/Validation'

export const checkLength = (o: {password?: string}, minLength: number = 6) => {
  const {password} = o
  return (!password || password.length < minLength) ?
    new Failure(['Password must have more than 6 characters']) : new Success(o)
}
```        

##### src/1104/test/checkPassword-test.ts

```ts
import {checkPassword} from '../checkPassword'

[ 
  {password: '123456'}, // 0 validation ok. {"password":"123456"}
  {password: '1234'}, // 1 validation fail. ["Password must have more than 6 characters"]
  {}, /* 2 validation fail. ["Password can not be null","Password must have more than 6 characters"] */
  {pa: '123456'} /* 3 validation fail. ["Password can not be null","Password must have more than 6 characters"] */
]
  .forEach((target, index) => {
    const [value, failureReson] = checkPassword(target)
    if(failureReson)
      console.log(index, 'validation fail.', JSON.stringify(failureReson))
    else
      console.log(index, 'validation ok.', JSON.stringify(value))
  })
```        
#### 이메일 주소 검증 기능 구현

##### src/1104/utils/checkEmailAdress.ts

```ts
import {Success, Failure} from '../classes/Validation'

export const checkEmailAddress = (o: {email?: string}) => {
  const {email} = o
  const re = new RegExp(/^(([^<>()\[\]\\.,;:\s@"]+(\.[^<>()\[\]\\.,;:\s@"]+)*)|(".+"))@((\[[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}])|(([a-zA-Z\-0-9]+\.)+[a-zA-Z]{2,}))$/)
  return re.test(email) ? new Success(email) 
    : new Failure(['invalid email address'])
}
```    

##### src/1104/checkEmail.ts

```ts
import {Success, Failure} from '../classes/Validation'

export const checkEmailAddress = (o: {email?: string}) => {
  const {email} = o
  const re = new RegExp(/^(([^<>()\[\]\\.,;:\s@"]+(\.[^<>()\[\]\\.,;:\s@"]+)*)|(".+"))@((\[[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}])|(([a-zA-Z\-0-9]+\.)+[a-zA-Z]{2,}))$/)
  return re.test(email) ? new Success(email) 
    : new Failure(['invalid email address'])
}
```    
          

##### src/1104/test/checkEmail-test.ts

```ts
import {checkEmail} from '../checkEmail'

[
  {email: 'abc@efg.com'}, // 0 validation ok. {"email":"abc@efg.com"}
  {email: 'abcefg'}       // 1 validation fail. ["invalid email address"]
].forEach((target, index) => {
  const [value, failureReson] = checkEmail(target)
  if(failureReson)
    console.log(index, 'validation fail.', JSON.stringify(failureReson))
  else
    console.log(index, 'validation ok.', JSON.stringify(value))
})
```    

### 11.5 IO 모나드 이해와 구현

#### IO 모나드란?
* Promise타입 객체는 생성할 떄 넘겨 주는 콜백함수가 then함수를 호출해야 동작하는데 
* IO모나드가 이런방식으로 동작한다.

##### src/1105/test/run-test.ts

```ts
import {IO} from '../classes/IO'

const work = () => {
  console.log('work called...')
  return {name: "Jack", age: 32}
}
const result = IO.of(work).runIO()
console.log(result)
```

##### src/1105/test/lens-test.ts

```ts
import { IO } from '../classes/IO'
import * as R from 'ramda'

const work = () => ({ name: 'Jack', age: 32 })

const result = IO.of(work)
  // @ts-ignore
  .map(R.view(R.lensProp('name')))
  .map(R.toUpper)
  .runIO()
console.log(result) // JACK
```

##### src/1105/test/chain-test.ts

```ts
import * as R from 'ramda'
import {IO} from '../classes/IO'

const work1 = () => ({name: "Jack"})
const work2 = (obj) => () => ({age: 32, ...obj})

const obj = IO.of(work1)
  .chain( (obj: object) => IO.of(work2(obj)) )
  .runIO()
console.log(obj) // { age: 32, name: 'Jack' }
```

#### 왜 모나드 이름이 IO인간?
* IO모나드는 여러개의 파일 입출력을 선언형 프로그래밍방식으로 작성할 수 있게 고안되었다.

##### src/1105/test/merge-file-test.ts

```ts
import * as R from 'ramda'
import * as fs from 'fs'
import {IO} from '../classes/IO'

const work1 = () => fs.readFileSync('package.json')
const work2 = (json1) => () => {
  const json2 = fs.readFileSync('tsconfig.json')
  return [json1, json2]
}

const result = IO.of(work1)
  .chain(json1 => IO.of(work2(json1)))
  .map(R.map(JSON.parse))
  .map(R.reduce((result: object, obj: object) => ({...result, ...obj}), {}))
  .runIO()
console.log(result)
```

#### IO모나드를 사용할 때 주의할 점
* 함수형 프로그래밍을 할 때 함수가 순수함수이어야 하지만 
* 비동기입출력, 프로미스, 생성기등은 부수효과가 발생하는 함수를 만들어 버린다.
* 따라서, fs.readFile과 같은 비동기 함수가 아니라 동기 함수인 fs.readFileSync함수를 사용

#### runIO메서드 이해하기

##### src/1105/classes/IRunIO.ts
* IO모나드의 runIO메서드는 여러 개의 매개변수를 사용해 동작할 수 있다.

```ts
export interface IRunIO {
  runIO<R>(...args: any[]): R
}

import {IO} from '../classes/IO'

const work = (a:number, b:number) => a + b;
const result = IO.of(work).runIO(1, 2);
console.log(result);  // 3
```

#### IO 모나드 구현
* IO모나드 상세설명 참고

##### src/1105/classes/IO.ts
* IO모나드의 runIO메서드는 여러 개의 매개변수를 사용해 동작할 수 있다.

```ts
import {IRunIO} from './IRunIO'
import {IFunctor} from '../interfaces'

const pipe = (...funcs) => (arg) => funcs.reduce((value, fn) => fn(value), arg)

export class IO implements IRunIO, IFunctor<Function>  {
  constructor(public fn: Function) {}
  static of(fn: Function) { return new IO(fn)}

  // IRunIO
  runIO<T>(...args: any[]): T {
    return this.fn(...args) as T
  }

  // IFunctor
  map(fn: Function): IO {
    const f: Function = pipe(this.fn, fn) 
    return IO.of(f)
  }

  // IChain
  chain(fn){
    const that = this
    return IO.of((value) => {
      const io = fn(that.fn(value))
      return io.fn()
    })
  }
}
```

#### 앞 메서드들의 반환값 얻기

##### src/1105/test/step1.ts
* a1변수는 runIO가 전다해 준 시작값

```ts
import {IO} from '../classes/IO'

const result = IO.of((a1) => {
  console.log('io started', a1)
  return a1
})
.runIO(1)
console.log(result) // 1
```

##### src/1105/test/step2.ts
* IO객체의 콜백함수는 a1변수값을 반환하는데 map메서드는 이 값을 다른 모나드와 동일방식으로 얻는다.

```ts
import {IO} from '../classes/IO'

const result = IO.of((a1) => {
  console.log('io started', a1)
  return a1
})
.map(a2 => {
  console.log('first map called', a2)
  return a2 + 1
})
.runIO(1)
console.log(result) // 2
```

##### src/1105/test/step3.ts
* 그러나 chain메서드 일 때는 IO모나드를 반환해야 한다.

```ts
import {IO} from '../classes/IO'

const result = IO.of((a1) => {
  console.log('io started', a1)
  return a1
})
.chain(a2 => {
  return IO.of(() => {
    console.log('first chain called', a2)
    return a2 + 1
  })
  
})
.runIO(1)
console.log(result) // 2
```

##### src/1105/test/step4.ts
* 결론적으로 chain메서드에서 이전 작업의 결과를 얻으려면 chainCB처럼 마치 
* 2차 고차함수형태로 보이는 방식으로 구현해야 한다.
```ts
import {IO} from '../classes/IO'

const chainCB = a2 => IO.of(() => a2 + 1)

const result = IO.of((a1) => {
  console.log('io started', a1)
  return a1
})
.chain(chainCB)
.runIO(1)
console.log(result) // 2
```

####  IO 모나드 상세설명

📌 IO 모나드란?
```
* IO 모나드는 부작용(Side Effect)을 함수형프로그래밍에서 안전하게 다루기 위한 개념
* 입출력(IO, Input/Output) 연산은 부작용을 일으키지만, 
* 이를 순수 함수처럼 다룰 수 있도록 래핑(wrapping) 합니다.
* IO 객체는 계산을 즉시 실행하지 않고, "나중에 실행될 함수(fn)를 감싸서" 보관.
* .runIO()를 호출해야만 실행되므로, 실행 시점을 제어할 수 있습니다.
```

🚀 코드와 설명
```ts
import { IRunIO } from './IRunIO'
import { IFunctor } from '../interfaces'

// 📌 pipe: 여러 함수를 합성하는 유틸리티 함수
const pipe = (...funcs) => (arg) => funcs.reduce((value, fn) => fn(value), arg)

// 📌 IO 모나드 클래스
export class IO implements IRunIO, IFunctor<Function> {
  
  // 🔹 IO 모나드는 부작용을 감싸기 위해 함수를 보관함
  constructor(public fn: Function) {}

  // 🔹 `of` 함수 (팩토리 메서드)
  // IO 모나드를 안전하게 생성하는 정적 메서드
  static of(fn: Function) { return new IO(fn) }

  // 🔹 IRunIO 인터페이스 구현
  // 실제 감싸져 있는 함수를 실행하는 메서드
  runIO<T>(...args: any[]): T {
    return this.fn(...args) as T
  }

  // 🔹 IFunctor 인터페이스 구현
  // `map` 메서드는 기존 IO 모나드의 값을 변경하지만 즉시 실행하지 않음
  map(fn: Function): IO {
    // pipe를 이용해 기존 `fn`과 새로운 `fn`을 합성
    const f: Function = pipe(this.fn, fn)
    return IO.of(f) // 새로운 IO 인스턴스를 반환
  }

  // 🔹 IChain (FlatMap 개념)
  // IO 모나드 내의 값을 IO 컨텍스트를 유지한 채 변경
  chain(fn) {
    const that = this
    return IO.of((value) => {
      const io = fn(that.fn(value)) // `fn`을 실행하면 새로운 IO 모나드가 나옴
      return io.fn() // 내부의 함수를 실행하여 최종 값을 반환
    })
  }
}
```

🔍 주요 개념 설명

```ts
// 1️⃣ of 정적 메서드
static of(fn: Function) { return new IO(fn) }
// 역할: 값을 IO 모나드로 감싸는 역할 (모나드의 unit 함수).
// 이점: 부작용을 직접 실행하지 않고 나중에 실행할 수 있도록 래핑.

// 📌 예제
const getUserInput = IO.of(() => prompt("Enter something: "));
// prompt()를 즉시 실행하지 않고, IO 객체 내부에 감싸서 보관.

// 2️⃣ runIO 메서드
runIO<T>(...args: any[]): T {
  return this.fn(...args) as T
}
// 역할: 감싸둔 함수를 실제 실행하는 역할.
// IO 모나드가 실행 시점을 제어하는 핵심적인 기능.
// runIO()를 호출해야만 부작용이 실행됨.
// 📌 예제
const io = IO.of(() => "Hello, World!");
console.log(io.runIO()); // "Hello, World!"
// IO 객체를 생성한 후 실행하기 전까지는 "Hello, World!"가 출력되지 않음.

// 3️⃣ map 메서드 (Functor)
map(fn: Function): IO {
  const f: Function = pipe(this.fn, fn);
  return IO.of(f);
}
// 역할: IO 모나드 안의 값을 변경하지만 즉시 실행하지 않음.
// Functor 규칙: map을 적용하면 새로운 IO 모나드가 반환됨.
// pipe를 사용해 기존 함수와 새로운 함수를 합성하여 연결.
// 📌 예제
const io = IO.of(() => 10);
const newIO = io.map(x => x * 2);
console.log(newIO.runIO()); // 20
// 원본 IO는 그대로 유지되며, map을 통해 새로운 IO가 생성됨.

// 4️⃣ chain 메서드 (Monad - FlatMap)/
chain(fn) {
  const that = this;
  return IO.of((value) => {
    const io = fn(that.fn(value));
    return io.fn();
  });
}
// 역할: 중첩된 IO 모나드를 평평하게(flatten) 만듦.
// 모나드 규칙: chain을 적용하면 IO(IO(value))가 아닌 IO(value)로 변환됨.
// fn이 반환하는 값이 또 다른 IO 객체일 때, .map()과 다르게 내부의 함수를 실행해서 // 값을 꺼냄.
// 📌 예제
const readInput = IO.of(() => "Hello");
const toUpperCase = (str) => IO.of(() => str.toUpperCase());

const result = readInput.chain(toUpperCase);
console.log(result.runIO()); // "HELLO"
// toUpperCase는 IO 객체를 반환하므로, .map() 대신 .chain()을 사용해야 함.
// .map()을 사용하면 IO(IO("HELLO"))가 되지만, .chain()을 사용하면 IO("HELLO")가 됨.
// 🚀 전체 동작 예제
const getUserInput = IO.of(() => prompt("Enter your name: "));
const toGreeting = (name) => IO.of(() => `Hello, ${name}!`);

const finalIO = getUserInput.chain(toGreeting);

// 사용자 입력을 받기 전까지 실행되지 않음
console.log(finalIO.runIO());

// 📌 실행 흐름
// 1. IO.of(() => prompt("Enter your name: ")) → 프롬프트를 감싼 IO 객체 생성.
// 2. .chain(toGreeting) → 입력값을 받아 Hello, {name}!을 만드는 새로운 IO 객체 생성.
// 3. .runIO()를 호출하면 모든 계산이 수행되고 최종 결과 반환.
```
## 📌 정리

| 메서드      | 역할                  | 반환 타입      | 설명 |
|------------|----------------------|--------------|------|
| `of(fn)`   | 값을 IO 모나드로 감싸기 | `IO<Function>` | 부작용을 감싸서 즉시 실행되지 않게 함 |
| `runIO()`  | 내부 함수 실행         | `T`          | IO 내부의 함수를 실행하여 실제 값을 반환 |
| `map(fn)`  | Functor 연산           | `IO<Function>` | IO 내부 값을 변환하지만 실행은 나중에 함 |
| `chain(fn)`| Monad 연산             | `IO<T>`       | 중첩된 IO를 평평하게 만들어 실행 가능하게 함 |

## 🚀 결론
```
* IO 모나드는 부작용을 안전하게 감싸고 실행 시점을 제어할 수 있도록 도와줍니다.
* .map()을 사용하면 연산을 조합할 수 있지만 실행하지 않습니다.
* .chain()을 사용하면 IO 내부의 값을 추출하여 실행 가능합니다.
* runIO()를 호출할 때만 실행되므로, 순수 함수형 스타일을 유지하면서도 필요할 때 실행할 수 있습니다.
```